# 6. National Forest Inventory (18 points) [Christian]

In [ ]:
import os

filename = 'NFI_filtered_cleaned.csv'
filepath = os.path.join('data', filename)

## Download data

In [ ]:
import os
import requests

url_data = 'https://raw.githubusercontent.com/christian-igel/ML/main/data/' + filename

if not os.access(filepath, os.R_OK):
    print('downloading', url_data)
    try:
        r = requests.get(url_data)
        r.raise_for_status()
        open(filepath, 'wb').write(r.content)
    except Exception as e:
        print('download failed:', e)

downloading https://raw.githubusercontent.com/christian-igel/ML/main/data/NFI_filtered_cleaned.csv


## Read data

In [ ]:
import pandas as pd

df = pd.read_csv(filepath)
df.to_csv(filepath, index=False)
df.describe()

,h_mean_1_,h_mean_2_,h_std_1_,h_std_2_,h_coov_1_,h_coov_2_,h_kur_1_,h_kur_2_,h_skew_1_,h_skew_2_,...,red_q50,red_q25,blue_q75,blue_q50,blue_q25,green_q75,green_q50,green_q25,BMag_ha,C_frac
count,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,...,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000,3156.000000
mean,7.409585,10.662752,5.105565,3.577681,1.051682,0.368356,1.838990,0.649883,0.288754,-0.315015,...,22139.944233,18636.451204,22296.699620,19473.237009,16995.629911,25513.125475,21920.365019,18655.026616,113.640865,0.468314
std,5.411430,5.781779,2.692018,1.783427,0.852839,0.140408,17.111033,4.515893,1.561340,0.909943,...,8326.425086,8049.423340,7008.356714,6988.936708,6880.897220,7293.151396,7354.904510,7232.853660,104.941748,0.499074
min,0.007347,1.117059,0.071730,0.000000,0.075114,0.000000,-1.926685,-3.000000,-13.816676,-3.747779,...,0.000000,0.000000,256.000000,0.000000,0.000000,256.000000,0.000000,0.000000,0.002059,0.000000
25%,2.906589,5.994878,3.073276,2.330300,0.567644,0.272190,-1.277054,-0.690741,-0.627047,-0.845669,...,16384.000000,12800.000000,17600.000000,14848.000000,12544.000000,20224.000000,16640.000000,13568.000000,28.934917,0.000000
50%,6.438852,10.085121,4.823045,3.371879,0.835525,0.352517,-0.623700,-0.122410,0.095032,-0.389580,...,22016.000000,18432.000000,22016.000000,19200.000000,16640.000000,25344.000000,21760.000000,18688.000000,86.230206,0.000000
75%,10.894731,14.578052,6.953171,4.658597,1.272893,0.435651,1.014367,0.883007,0.947310,0.161931,...,28160.000000,24320.000000,26368.000000,23552.000000,20992.000000,30272.000000,26880.000000,23552.000000,169.278397,1.000000
max,31.577832,31.654262,14.545132,12.496709,13.132664,1.359926,573.404330,205.440694,18.965810,7.757708,...,51200.000000,51200.000000,53376.000000,50176.000000,50176.000000,53376.000000,47104.000000,44288.000000,956.210950,1.000000


## Define relevant features

In [ ]:
regression_target = 'BMag_ha'
classification_target = 'C_frac'
features = ['h_mean_1_', 'h_mean_2_', 'h_std_1_', 'h_std_2_', 'h_coov_1_', 'h_coov_2_', 'h_skew_1_', 'h_skew_2_',
            'IR_', 'h_q5_1_', 'h_q10_1_', 'h_q25_1_', 'h_q50_1_', 'h_q75_1_', 'h_q90_1_', 'h_q95_1_', 'h_q99_1_', 'h_q5_2_',
            'h_q10_2_', 'h_q25_2_', 'h_q50_2_', 'h_q75_2_', 'h_q90_2_', 'h_q95_2_', 'h_q99_2_', 'red_q75', 'red_q50', 'red_q25',
            'blue_q75', 'blue_q50', 'blue_q25', 'green_q75', 'green_q50', 'green_q25']

## Combine training- and validation set into a single data frame, for tasks that don't require a validation set.

In [ ]:
# Define set with train and validation data
df_trainval = df.query("split in ['train', 'val']")

# Define set with test data
df_test = df.query("split == 'test'")

# Select only the features we want to use
X_trainval = df_trainval[features]
X_test = df_test[features]

# Convert target values from dataframes to numpy arrays
y_trainval = df_trainval[[regression_target]].values.ravel()
y_test = df_test[[regression_target]].values.ravel()

## 1. Linear Regression without regularization. Report the test $R^2$ score.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Define linear regression model
lr_model = LinearRegression()

# Fit model on TrainVal data
lr_model.fit(X_trainval, y_trainval)

# Predict on TrainVal and Test data
lr_trainval_y_pred = lr_model.predict(X_trainval)
lr_test_y_pred = lr_model.predict(X_test)

# Print R2 scores
print(f"R2 score (training): {r2_score(y_trainval, lr_trainval_y_pred):.3f}")
print(f"R2 score (test): {r2_score(y_test, lr_test_y_pred):.3f}")

R2 score (training): 0.712
R2 score (test): 0.739


## 2. Random forest with 100 and 1000 trees. Use the OOB score to determine which number of trees is better. Report OOB scores of both models and the test $R^2$ score of the better solution.

### 100 trees

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Define parameter grid
rf_param_grid = {
    'n_estimators': [100, 1000]
}

# Define random forest model
rf_model = RandomForestRegressor(random_state=0, oob_score=True)

# Grid search
rf_grid_search = GridSearchCV(rf_model, param_grid=rf_param_grid, cv=5, n_jobs=-1)

# Fit model with grid search on TrainVal data
rf_grid_search.fit(X_trainval, y_trainval)

# Best model from grid search
rf_best_model = rf_grid_search.best_estimator_

# Predict with best model on TrainVal and Test data
rf_best_model_trainval_y_pred = rf_best_model.predict(X_trainval)
rf_best_model_test_y_pred = rf_best_model.predict(X_test)

# Print best number of estimators
print(f"Best number of estimators: {rf_grid_search.best_params_['n_estimators']}")
# Print OOB score
print(f"OOB score: {rf_best_model.oob_score_}")
# Print R2 scores
print(f"R2 score (training): {r2_score(y_trainval, rf_best_model_trainval_y_pred):.3f}")
print(f"R2 score (test): {r2_score(y_test, rf_best_model_test_y_pred):.3f}")

## k-nearest-neighbor regression. Find out if normalization to zero mean and unit variance helps. Describe how you did that. Use cross-validation to determine the number of neighbors from the range {1, 3, 5, ..., 47, 49}. Describe how you determined the number of neighbors. Report training and test $R^2$ score of the best model.

### Unnormalized

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score

# Define parameter grid
knn_param_grid = {'n_neighbors': list(range(1, 50, 2))} # odd numbers from 1 to 50 (majority vote, no ties)

# Define kNN model and grid search
knn_grid_search = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=knn_param_grid, cv=5, n_jobs=-1)

# Fit model with grid search on training set
knn_grid_search.fit(X_trainval, y_trainval)

# Best model from grid search
knn_best_unnormalized_model = knn_grid_search.best_estimator_

# Predict with best model on TrainVal and Test data
knn_best_unnormalized_model_trainval_y_pred = knn_best_unnormalized_model.predict(X_trainval)
knn_best_unnormalized_model_test_y_pred = knn_best_unnormalized_model.predict(X_test)

# Print best number of neighbors
print(f"Best number of neighbors: {knn_grid_search.best_params_['n_neighbors']}")
# Print R2 score for training set and test set
print(f"R2 score (training): {r2_score(y_trainval, knn_best_unnormalized_model_trainval_y_pred)}")
print(f"R2 score (test): {r2_score(y_test, knn_best_unnormalized_model_test_y_pred)}")

Best number of neighbors: 33
R2 score (training): 0.1345567872393415
R2 score (test): 0.02687036261417819


### Normalized

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score
from sklearn.preprocessing import StandardScaler

# Assuming X_train and X_test are your training and test datasets
scaler = StandardScaler()
X_trainval_normalized = scaler.fit_transform(X_trainval)
X_test_normalized = scaler.transform(X_test)

# Define parameter grid
knn_param_grid = {'n_neighbors': list(range(1, 50, 2))}  # {1, 3, 5, ..., 49}

# Define kNN model
knn_model = KNeighborsRegressor()

# Grid search
knn_grid_search = GridSearchCV(estimator=knn_model, param_grid=knn_param_grid, cv=5, n_jobs=-1)

# Fit model with grid search on training set
knn_grid_search.fit(X_trainval_normalized, y_trainval)

# Best model from grid search
knn_best_normalized_model = knn_grid_search.best_estimator_

# Predict with best model on test set
knn_best_normalized_model_y_pred = knn_best_normalized_model.predict(X_test_normalized)

# Print best number of neighbors
print(f"Best number of neighbors: {knn_grid_search.best_params_['n_neighbors']}")
# Print R2 score for training set and test set
print(f"R2 score (training): {r2_score(y_trainval, knn_best_normalized_model.predict(X_trainval_normalized))}")
print(f"R2 score (test): {r2_score(y_test, knn_best_normalized_model_y_pred)}")

Best number of neighbors: 41
R2 score (training): 0.705935005362279
R2 score (test): 0.7316752547056613


# 7. Clustering (20 points) [Sadegh]

In [ ]:
import numpy as np
from tabulate import tabulate

def kmeans(data, initial_centroids):
    centroids = np.array([data[point] for point in initial_centroids])
    prev_centroids = np.zeros(centroids.shape)
    clusters = {i: [] for i in range(len(centroids))}
    iteration = 1
    
    # Continue until centroids converge
    while not np.array_equal(centroids, prev_centroids):
        print(f"\nIteration {iteration}:\n{'-' * 20}")
        clusters = {i: [] for i in range(len(centroids))}

        # Assignment step
        table_data = []
        for point_name, point_coords in data.items():
            distances = np.linalg.norm(point_coords - centroids, axis=1)
            cluster_index = np.argmin(distances)
            clusters[cluster_index].append(point_name)
            table_data.append([point_name] + list(distances))
        headers = ["Point"] + [f"c{i+1}" for i in range(len(centroids))]
        print(tabulate(table_data, headers=headers, floatfmt=".3f"))        

        # Update step
        prev_centroids = centroids.copy()
        for i, point_names in clusters.items():
            if point_names:
                new_centroid = np.mean([data[pn] for pn in point_names], axis=0)
                centroids[i] = new_centroid
                print(f"Updated cluster center, c{i+1}: from [{prev_centroids[i][0]:.3f}, "
                      f"{prev_centroids[i][1]:.3f}] to [{new_centroid[0]:.3f}, {new_centroid[1]:.3f}]")

        for i, point_names in clusters.items():
            print(f"Cluster c{i+1}: {point_names}")
        
        iteration += 1
    
    # Display final results
    print("\nFinal Results:\n" + '-' * 20)
    for i, point_names in clusters.items():
        print(f"Cluster c{i+1}: {point_names}")
    for i, centroid in enumerate(centroids):
        formatted_centroid = [f"{element:.3f}" for element in centroid]
        print(f"c{i+1} = [{', '.join(formatted_centroid)}]")

In [ ]:
# Data points
data = {
    'A': np.array([25., 0.]),
    'B': np.array([0., 3.]),
    'C': np.array([0., -3.]),
    'D': np.array([-25., 1.]),
    'E': np.array([-25., -1.]),
}

## 7.4

In [ ]:
initial_centroids = ['C', 'A', 'D']
kmeans(data, initial_centroids)


Iteration 1:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        25.179   0.000  50.010
B         6.000  25.179  25.080
C         0.000  25.179  25.318
D        25.318  50.010   0.000
E        25.080  50.010   2.000
Updated cluster center, c1: from [0.000, -3.000] to [0.000, 0.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-25.000, 1.000] to [-25.000, 0.000]
Cluster c1: ['B', 'C']
Cluster c2: ['A']
Cluster c3: ['D', 'E']

Iteration 2:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        25.000   0.000  50.000
B         3.000  25.179  25.179
C         3.000  25.179  25.179
D        25.020  50.010   1.000
E        25.020  50.010   1.000
Updated cluster center, c1: from [0.000, 0.000] to [0.000, 0.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-25.000, 0.000] to [-25.000, 0.000]
Cluster c

## 7.5

In [ ]:
# Initial centroids
initial_centroids = ['D', 'A', 'E']
kmeans(data, initial_centroids)


Iteration 1:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        50.010   0.000  50.010
B        25.080  25.179  25.318
C        25.318  25.179  25.080
D         0.000  50.010   2.000
E         2.000  50.010   0.000
Updated cluster center, c1: from [-25.000, 1.000] to [-12.500, 2.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-25.000, -1.000] to [-12.500, -2.000]
Cluster c1: ['B', 'D']
Cluster c2: ['A']
Cluster c3: ['C', 'E']

Iteration 2:
--------------------
Point        c1      c2      c3
-------  ------  ------  ------
A        37.553   0.000  37.553
B        12.540  25.179  13.463
C        13.463  25.179  12.540
D        12.540  50.010  12.855
E        12.855  50.010  12.540
Updated cluster center, c1: from [-12.500, 2.000] to [-12.500, 2.000]
Updated cluster center, c2: from [25.000, 0.000] to [25.000, 0.000]
Updated cluster center, c3: from [-12.500, -2.000] to [-12.500, -2.000